In [1]:
#import data form csv file 
import csv

dates = []
columnNames = []
signal = []
spy = []

i=1
#Cannot disclose the confidential dataset. Will prepare a testing data soon from public source
with open('data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if i==1: 
            i=i+1
            columnNames = row
            continue            
        if row[0]: 
            dates.append(row[0])
            signal.append(row[1])  
            spy.append(row[2]) 

In [2]:
#convert data string to float
import numpy as np
numRow = len(spy)
dataAll = np.zeros((numRow,2))
for i in range(numRow):
        if len(signal[i]) != 0: #empty entry has a default value of 0.00
            dataAll[i,0] = float(signal[i])
        if len(spy[i]) != 0: #empty entry has a default value of 0.00
            dataAll[i,1] = float(spy[i])
            
#convert dates to datetime.dates
import datetime as dt
datesData = np.array([dt.datetime.strptime(d,'%Y%m%d').date() for d in dates])

In [3]:
#plot data and check by eyes
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(1)
subfig1 = plt.subplot(211)
plt.plot(datesData, dataAll[:,0], color='blue',linewidth=3)
subfig1.set_title("Signal")

subfig2 = plt.subplot(212)
plt.plot(datesData, dataAll[:,1], color='red', linewidth=3)
subfig2.set_title("SPY")

plt.show()

In [4]:
#plot data and check by eyes
plt.figure(1)
plt.scatter(dataAll[:,0],  dataAll[:,1], color='red')
plt.show()

In [5]:
plt.figure(1)
plt.subplot(211)
plt.hist(dataAll[:,0])

plt.subplot(212)
plt.hist(dataAll[:,1])
plt.show()

there could be outliers or error data 

In [6]:
#identify outliers with MLE which assuming both signal and spy with normal distribution. 
#The assumption may not be sure and works well for the case.
import numpy as np
import matplotlib.pyplot as plt
from sklearn.covariance import EmpiricalCovariance

n_features = 2
X = dataAll

#estimators learnt from the full data set with true parameters
emp_cov = EmpiricalCovariance().fit(X)

###############################################################################
# Display results
fig = plt.figure()

# Show data set
subfig1 = plt.subplot(211)
inlier_plot = subfig1.scatter(X[:, 0], X[:, 1],
                              color='black', label='inliers')
subfig1.set_title("Signal vs SPY")

# Show contours of the distance functions
xx, yy = np.meshgrid(np.linspace(plt.xlim()[0], plt.xlim()[1], 100),
                     np.linspace(plt.ylim()[0], plt.ylim()[1], 100))
zz = np.c_[xx.ravel(), yy.ravel()]

mahal_emp_cov = emp_cov.mahalanobis(zz)
mahal_emp_cov = mahal_emp_cov.reshape(xx.shape)
emp_cov_contour = subfig1.contour(xx, yy, np.sqrt(mahal_emp_cov),
                                  cmap=plt.cm.PuBu_r,
                                  linestyles='dashed')


plt.xticks(())
plt.yticks(())

# Plot the scores for each point
emp_mahal = emp_cov.mahalanobis(X - np.mean(X, 0)) ** (0.33)
subfig2 = plt.subplot(212)
bp = subfig2.boxplot(emp_mahal)
subfig2.plot(1.26 * np.ones(numRow),
             emp_mahal[:], '+k', markeredgewidth=1)

#subfig2.axes.set_xticklabels(('inliers', 'outliers'), size=15)
subfig2.set_ylabel(r"$\sqrt[3]{\rm{(Mahal. dist.)}}$", size=16)
subfig2.set_title("1. from non-robust estimates\n(Maximum Likelihood)")
plt.yticks(())
plt.yticks(())

limit = bp["caps"][1].get_data()[1][0]
outliers = X[np.where(emp_mahal>limit)]
outlier_plot = subfig1.scatter(outliers[:, 0], outliers[:, 1], color='red', label='outliers')

subfig1.legend([emp_cov_contour.collections[1],
                inlier_plot, outlier_plot],
               ['MLE dist', 'inliers', 'outliers'],
               loc="upper right", borderaxespad=0)

plt.show()

In [7]:
#remove outlier from dataAll
dataNew = dataAll[np.where(emp_mahal<limit)]
datesDataNew = datesData[np.where(emp_mahal<limit)]


In [8]:
import datetime as dt
import numpy as np
datesData = np.array([dt.datetime.strptime(d,'%Y%m%d').date() for d in dates])
datesDataNew = datesData[np.where(emp_mahal<limit)]
#plot data and check by eyes
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(1)
subfig1 = plt.subplot(211)
plt.plot(datesDataNew, dataNew[:,0], color='blue',linewidth=3)
subfig1.set_title("Signal")

subfig2 = plt.subplot(212)
plt.plot(datesDataNew, dataNew[:,1], color='red', linewidth=3)

subfig2.set_title("SPY")

plt.show()

In [9]:
#plot data and check by eyes
import matplotlib.pyplot as plt

plt.figure(1)
plt.scatter(dataNew[:,0],dataNew[:,1], color='red')
plt.show()

more outliers may exist beased on observation

In [10]:
#run the same algorithm and identify other outliers

import numpy as np
import matplotlib.pyplot as plt
from sklearn.covariance import EmpiricalCovariance, MinCovDet

n_features = 2
X = dataNew
numRow = len(X)

# compare estimators learnt from the full data set with true parameters
emp_cov = EmpiricalCovariance().fit(X)

###############################################################################
# Display results
fig = plt.figure()

# Show data set
subfig1 = plt.subplot(211)
inlier_plot = subfig1.scatter(X[:, 0], X[:, 1],
                              color='black', label='inliers')
subfig1.set_title("Signal vs SPY")

# Show contours of the distance functions
xx, yy = np.meshgrid(np.linspace(plt.xlim()[0], plt.xlim()[1], 100),
                     np.linspace(plt.ylim()[0], plt.ylim()[1], 100))
zz = np.c_[xx.ravel(), yy.ravel()]

mahal_emp_cov = emp_cov.mahalanobis(zz)
mahal_emp_cov = mahal_emp_cov.reshape(xx.shape)
emp_cov_contour = subfig1.contour(xx, yy, np.sqrt(mahal_emp_cov),
                                  cmap=plt.cm.PuBu_r,
                                  linestyles='dashed')

plt.xticks(())
plt.yticks(())

# Plot the scores for each point
emp_mahal = emp_cov.mahalanobis(X - np.mean(X, 0)) ** (0.33)
subfig2 = plt.subplot(212)
bp = subfig2.boxplot(emp_mahal)
subfig2.plot(1.26 * np.ones(numRow),
             emp_mahal[:], '+k', markeredgewidth=1)

#subfig2.axes.set_xticklabels(('inliers', 'outliers'), size=15)
subfig2.set_ylabel(r"$\sqrt[3]{\rm{(Mahal. dist.)}}$", size=16)
subfig2.set_title("1. from non-robust estimates\n(Maximum Likelihood)")
plt.yticks(())
plt.yticks(())

limit = bp["caps"][1].get_data()[1][0]
outliers = X[np.where(emp_mahal>limit)]
outlier_plot = subfig1.scatter(outliers[:, 0], outliers[:, 1], color='red', label='outliers')
subfig1.legend([emp_cov_contour.collections[1],
                inlier_plot, outlier_plot],
               ['MLE dist', 'inliers', 'outliers'],
               loc="upper right", borderaxespad=0)

plt.show()

In [11]:
#remove more outliers
dataNew2 = dataNew[np.where(emp_mahal<limit)]

In [14]:
#evaluate what technique should be used
import datetime as dt
import numpy as np
#datesData = np.array([dt.datetime.strptime(d,'%Y%m%d').date() for d in dates])
datesDataNew2 = datesDataNew[np.where(emp_mahal<limit)]
#plot data and check by eyes
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(1)
subfig1 = plt.subplot(211)
plt.plot(datesDataNew2, dataNew2[:,0], color='blue',linewidth=3)
subfig1.set_title("Signal")

subfig2 = plt.subplot(212)
plt.plot(datesDataNew2, dataNew2[:,1], color='red', linewidth=3)

subfig2.set_title("SPY")

plt.show()

In [15]:
#plot data and check by eyes
import matplotlib.pyplot as plt
#lag =1
plt.figure(1)
#plt.scatter(dataNew2[lag:,0],dataNew2[:0-lag,1], color='red')
plt.scatter(dataNew2[:,0],dataNew2[:,1], color='red')
plt.show()



In [16]:
plt.figure(1)
plt.subplot(211)
plt.hist(dataNew2[:,0])

plt.subplot(212)
plt.hist(dataNew2[:,1])
plt.show()

In [18]:
retData = np.log(dataNew2[1:,:]/dataNew2[0:-1,:])

In [29]:
plt.figure(1)
subfig1 = plt.subplot(211)
plt.plot(datesDataNew2[1:], retData[:,0], color='blue',linewidth=3)
subfig1.set_title("Signal")

subfig2 = plt.subplot(212)
plt.plot(datesDataNew2[1:], retData[:,1], color='red', linewidth=3)

subfig2.set_title("SPY")

plt.show()

In [20]:
plt.figure(1)
plt.subplot(211)
plt.hist(retData[:,0])

plt.subplot(212)
plt.hist(retData[:,1])
plt.show()

In [30]:
#plot data and check by eyes
import matplotlib.pyplot as plt
#lag =6
plt.figure(1)
#plt.scatter(retData[lag:,0],retData[:0-lag,1], color='red')
plt.scatter(dataNew2[1:,0],retData[:,1], color='red')
plt.show()

very good linear correlation between signal and spy and I will try linear regression first.

In [113]:
#divide data into train and test dataset
# proportion of training data
p = 0.6
split = int(len(dataNew2)*0.6)
trainX = dataNew2[:,0][:split]
testX = dataNew2[:,0][split:]
trainY = dataNew2[:,1][:split]
testY = dataNew2[:,1][split:]
trainX =trainX.reshape(-1,1)
testX =testX.reshape(-1,1)

trainDate = datesDataNew2[:split]
testDate = datesDataNew2[split:]
trainDate =trainDate.reshape(-1,1)
testDate =testDate.reshape(-1,1)

In [114]:
#apply linear regression

import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model

# Create linear regression object
lm = linear_model.LinearRegression()
# Train the model using the training sets
lm.fit(trainX, trainY)
# The coefficients
print('Coefficients: \n', lm.coef_)
# The mean square error
print("Residual sum of squares: %.6f"
      % np.mean((lm.predict(testX) - testY) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.6f' % lm.score(testX, testY))

# Plot outputs
plt.figure(1)
subfig1 = plt.subplot(211)
subfig1.scatter(testX, testY,  color='red')
subfig1.scatter(testX, lm.predict(testX), color='blue')
subfig1.set_title("Signal")

subfig2 = plt.subplot(212)
subfig2.plot(range(0,len(testY)), lm.predict(testX), color='blue',
         linewidth=3)
subfig2.plot(range(0,len(testY)), testY, color='red',
         linewidth=3)

plt.show()


Coefficients: 
 [ 39.92454496]
Residual sum of squares: 7.108785
Variance score: 0.926325


R-squared is not high and out-of-sample testing is not good.

In [115]:
# Plot residuals and 
plt.plot(range(0,len(trainY)), trainY-lm.predict(trainX), color='red',
         linewidth=3)
plt.xticks(())
plt.yticks(())
plt.show()

In [116]:
#prepare residuals for applying pandas's time series analysis to get a hint for improving the linear model
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
import datetime
resTrain = trainY-lm.predict(trainX)
dateIndex = [datetime.datetime.combine(x[0], datetime.time.min)  for x in trainDate.tolist()]
dta = pd.DataFrame(resTrain, index=pd.Index(dateIndex), columns=['Residual'])
dta.plot(figsize=(12,8))
plt.show()

In [117]:
#check autocorrelation and partial autocorrelation of residuals
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(dta.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(dta, lags=40, ax=ax2)
plt.show()



Based on time series ananlysis, the model could be MA(3) or AR(3)

In [118]:
# test MA(3) case
trainX1 = np.roll(trainX,1)
trainX2 = np.roll(trainX,2)
trainX3 = np.roll(trainX,3)

trainXm = np.column_stack((trainX,trainX1,trainX2,trainX3))
trainXm = trainXm[3:,:]
trainYm = trainY[3:]

testX1 = np.roll(testX,1)
testX2 = np.roll(testX,2)
testX3 = np.roll(testX,3)

testXm = np.column_stack((testX,testX1,testX2,testX3))
testXm = testXm[3:,:]
testYm = testY[3:]

In [119]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model

# Create linear regression object
lm = linear_model.LinearRegression()
# Train the model using the training sets
lm.fit(trainXm, trainYm)

# The coefficients
print('Coefficients: \n', lm.coef_)
# The mean square error
print("Residual sum of squares: %.6f"
      % np.mean((lm.predict(testXm) - testYm) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.6f' % lm.score(testXm, testYm))

# Plot outputs
plt.figure(1)
plt.plot(range(0,len(testYm)), lm.predict(testXm), color='blue',
         linewidth=3)
plt.plot(range(0,len(testYm)), testYm, color='red',
         linewidth=3)
plt.show()



Coefficients: 
 [  2.32872012   0.98432396  24.56289895  12.67977699]
Residual sum of squares: 3.414044
Variance score: 0.964160


Adding MA(3) does not improve the model

In [120]:
#test AR(3) case
trainX1 = np.roll(trainY,1)
trainX2 = np.roll(trainY,2)
trainX3 = np.roll(trainY,3)

trainXm = np.column_stack((trainX,trainX1,trainX2,trainX3))
trainXm = trainXm[3:,:]
trainYm = trainY[3:]

testX1 = np.roll(testY,1)
testX2 = np.roll(testY,2)
testX3 = np.roll(testY,3)

testXm = np.column_stack((testX,testX1,testX2,testX3))
testXm = testXm[3:,:]
testYm = testY[3:]

In [121]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model

# Create linear regression object
lm = linear_model.LinearRegression()
# Train the model using the training sets
lm.fit(trainXm, trainYm)

# The coefficients
print('Coefficients: \n', lm.coef_)
# The mean square error
print("Residual sum of squares: %.6f"
      % np.mean((lm.predict(testXm) - testYm) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.6f' % lm.score(testXm, testYm))

# Plot outputs
plt.figure(1)
plt.plot(range(0,len(testYm)), lm.predict(testXm), color='blue',
         linewidth=3)
plt.plot(range(0,len(testYm)), testYm, color='red',
         linewidth=3)
plt.show()



Coefficients: 
 [ 10.31966367   0.75812301   0.01361146  -0.02607735]
Residual sum of squares: 1.338787
Variance score: 0.985946


AR(3) is a better model with significantly improved R2!

In [122]:
# Plot residuals for AR(3)
plt.plot(range(0,len(trainYm)), trainYm-lm.predict(trainXm), color='red',
         linewidth=3)
plt.xticks(())
plt.yticks(())
plt.show()

In [123]:
#since added three time lag terms, we tried other linear models to prevent overfitting.
from sklearn import linear_model
lmRidgecv = linear_model.RidgeCV(alphas=[0.1,0.5,1.0,5.0,10.0,25.0,50])
lmRidgecv.fit(trainXm, trainYm)       
#RidgeCV(alphas=[0.1, 1.0, 10.0], cv=None, fit_intercept=True, scoring=None,
#    normalize=False)
print("alpha: %d" % lmRidgecv.alpha_)
print("coef:" , lmRidgecv.coef_)
print("Residual sum of squares: %.6f"
      % np.mean((lmRidgecv.predict(testXm) - testYm) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.6f' % lmRidgecv.score(testXm, testYm))
#plt.scatter(testX, testY,  color='black')
plt.plot(range(len(testYm)), lmRidgecv.predict(testXm), color='blue',  linewidth=3)
plt.plot(range(len(testYm)), testYm, color='red',   linewidth=3)
plt.xticks(())
plt.yticks(())
plt.show()

alpha: 0
coef: [ 9.40097093  0.77805295  0.01344435 -0.0234138 ]
Residual sum of squares: 1.315673
Variance score: 0.986188


In [124]:
from sklearn import linear_model
clf = linear_model.LassoCV(alphas=[0.001,0.1,0.5,0.8,1.0,2.0,5.0,10.0,25.0])
clf.fit(trainXm, trainYm) 
print("alpha: %d" % clf.alpha_)
print("coef:" , clf.coef_)
print("Residual sum of squares: %.6f"
      % np.mean((clf.predict(testXm) - testYm) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.6f' % clf.score(testXm, testYm))
#plt.scatter(testX, testY,  color='black')
plt.plot(range(len(testYm)), clf.predict(testXm), color='blue',  linewidth=3)
plt.plot(range(len(testYm)), testYm, color='red',   linewidth=3)
plt.xticks(())
plt.yticks(())
plt.show()

alpha: 0
coef: [ 9.91475281  0.76783816  0.01185739 -0.02414552]
Residual sum of squares: 1.327455
Variance score: 0.986064


In [128]:

from sklearn import linear_model
clf = linear_model.ElasticNetCV(alphas=[0.001,0.1,0.5,0.8,1.0,2.0,5.0,10.0,25.0])
clf.fit(trainXm, trainYm) 
print("alpha: %d" % clf.alpha_)
print("coef:" , clf.coef_)
print("Residual sum of squares: %.6f"
      % np.mean((clf.predict(testXm) - testYm) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.6f' % clf.score(testXm, testYm))
#plt.scatter(testX, testY,  color='black')
plt.plot(range(len(testYm)), clf.predict(testXm), color='blue',  linewidth=3)
plt.plot(range(len(testYm)), testYm, color='red',   linewidth=3)
plt.xticks(())
plt.yticks(())
plt.show()


alpha: 0
coef: [ 8.49903788  0.79808296  0.01240818 -0.02038617]
Residual sum of squares: 1.299776
Variance score: 0.986355


All cross validaton tests lead to alpha = 0 for all three models supporting no overfitting. All models should be equally good.
The best model I choose in terms R-square is ElasticNet
# spy(t) = 8.499*Signal(t) + 0.7981*spy(t-1) 0.0124*spy(t-2) -0.0204*spy(t-2)


In [129]:
#Double check the final model's residual
#perform time series analysis on residuals
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
import datetime
resTrain = trainYm-clf.predict(trainXm)
dateIndex = [datetime.datetime.combine(x[0], datetime.time.min)  for x in trainDate.tolist()]
dta = pd.DataFrame(resTrain, index=pd.Index(dateIndex[3:]), columns=['Residual'])
dta.plot(figsize=(12,8))
plt.show()

In [130]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(dta.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(dta, lags=40, ax=ax2)
plt.show()

#Very good!
